<a href="https://colab.research.google.com/github/doukansurel/Kod/blob/main/Haberlercom_tamam%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [242]:
import os
import bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import time
import urllib.request
import re
import urllib3
from pandas import DataFrame
import csv
import datetime
from datetime import datetime, timedelta ,date
import concurrent

In [243]:
pages = []
pages_links = []
page_news_links = []

In [244]:
def date_creator():
  url = "https://www.haberler.com/arsiv/{}/haberler/s1/"
  num = 1
  t1 = datetime(2006,6,1)
  t2 = datetime(2021,8,5)                                                             # Genel liinkleri oluşturyor
  t= timedelta(days = 1)
  dates=np.arange(t1, t2, t).astype(datetime)
  for j in dates:
    new_url =(url.format(j.strftime('%d-%m-%Y'),num))
    pages.append(new_url)


In [245]:
date_creator()

In [246]:
def page_get_link(url):
  r = requests.get(url).content
  soup = BeautifulSoup(r, "html.parser")
  #p_count = soup.find("div",{"class":"hbPagination"}).find_all("span")
  array = []
  span = ""
  if not soup.find("div",{"class":"hbPagination"}) :                                # Linklerin sayfaları ile birlite ayarlıyor
    pages_links.append(url)
  else :
    p_count =soup.find("div",{"class":"hbPagination"}).find_all("span")
    for i in p_count :
      span = i.getText()
      array.append(span)

    page_num = len(array)-1
    new_url = url 
    new_url = new_url.replace("s1","s{}")

    for i in range(page_num):
      i = str(i+1)
      lnk = new_url.format(i)
      pages_links.append(lnk)


In [247]:
# for i in pages[:100]:
#   page_get_link(i)

In [204]:
# for i in pages_links[:100]:
#   print(i)

https://www.haberler.com/arsiv/01-06-2006/haberler/s1/
https://www.haberler.com/arsiv/01-06-2006/haberler/s2/
https://www.haberler.com/arsiv/01-06-2006/haberler/s3/
https://www.haberler.com/arsiv/01-06-2006/haberler/s4/
https://www.haberler.com/arsiv/02-06-2006/haberler/s1/
https://www.haberler.com/arsiv/02-06-2006/haberler/s2/
https://www.haberler.com/arsiv/02-06-2006/haberler/s3/
https://www.haberler.com/arsiv/03-06-2006/haberler/s1/
https://www.haberler.com/arsiv/03-06-2006/haberler/s2/
https://www.haberler.com/arsiv/04-06-2006/haberler/s1/
https://www.haberler.com/arsiv/04-06-2006/haberler/s2/
https://www.haberler.com/arsiv/05-06-2006/haberler/s1/
https://www.haberler.com/arsiv/05-06-2006/haberler/s2/
https://www.haberler.com/arsiv/05-06-2006/haberler/s3/
https://www.haberler.com/arsiv/06-06-2006/haberler/s1/
https://www.haberler.com/arsiv/06-06-2006/haberler/s2/
https://www.haberler.com/arsiv/06-06-2006/haberler/s3/
https://www.haberler.com/arsiv/07-06-2006/haberler/s1/
https://ww

In [ ]:
t1=time.time()
with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(page_get_link,i.strip()) for i in pages]
print(time.time()-t1)

In [248]:
def haber_get_link(url):
  r = requests.get(url).content
  soup = BeautifulSoup(r,"html.parser")
  list = soup.find_all("div",{"class":"box boxStyle color-general"})
                                                                           # tüm sayfalardaki haberlerin linklerini alıyor
  for i in list :
    href = i.find("a").get("href")
    page_news_links.append(href)
  
  with open ("haberlercom-Links.txt","a",encoding="utf-8") as file :
    for i in page_news_links:
      file.write(i+"\n")

In [249]:
# for i in pages_links[:50]:
#   haber_get_link(i)
t1=time.time()
with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(haber_get_link,i.strip()) for i in pages_links]
print(time.time()-t1)

In [250]:
def creator(url):
  r = requests.get(url).content
  soup = BeautifulSoup(r,"html.parser")
  title = soup.find("div",{"class":"hbptHead"}).find("h1",{"class":"haber_baslik"}).getText() 
  date = soup.find("div",{"class":"hbptHead hbdetaytarihCnt"}).find("time").getText()
  date =date[:15] 
  categori_array = []
  categori = soup.find("div",{"class":"hbptHead hbdetaytarihCnt"}).find_all("a")
  
  for i in categori :
    categori_array.append(i.getText())                                                      # Her Link içeriğini alır ve yazdırır
  cat = categori_array[len(categori_array)-1]  

  content_string = ""
  content = soup.find("div",{"class":"hbptContent haber_metni"}).find_all("p")
  
  for i in content:
    content_string += i.getText().strip("Kaynak: ")
  sentence = "{} ; {} ; {} ; {} ; {}".format(url,cat,date,title,content_string)

  with open("haberlercom-Cont.txt","a",encoding="utf-8")as file:
    file.write(sentence+"\n")


In [251]:
t1=time.time()
with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(creator,i.strip()) for i in page_news_links]
print(time.time()-t1)
# for i in page_news_links[:20]:
#   creator(i)